## Organizações não governamentais de ambiente por 100 000 habitantes (N.º) por Localização geográfica (NUTS - 2013); Anual
https://www.ine.pt/xportal/xmain?xpid=INE&xpgid=ine_indicadores&indOcorrCod=0008290&contexto=bd&selTab=tab2

### Base de dados: Nuts 2013

### Sinais convencionais
Sinais convencionais por ausência de valor
- `…`	Dado confidencial
- `-`	Dado nulo ou não aplicável
- `x`	Dado não disponível
- `o`	Dado inferior a metade do módulo da unidade utilizada
- `∞`	Infinito

Sinais convencionais adicionais ao valor
- `//` Dado preliminar
- `&`	Dado provisório
- `»`	Dado previsto
- `*`	Dado rectificado
- `§`	Desvio do padrão de qualidade/Coeficiente de variação elevado
- `“`	Estimativa
- `“E` Dado estimado pelo Eurostat
- `┴`	Quebra de série/comparabilidade
- `i`	Mais informação em anexo

## Sinais convencionais diversos
- `µ`	Média
- `=`	Igual
- `>`	Maior que
- `≥`	Maior ou igual
- `<`	Menor que
- `≤`	Menor ou igual
- `%`	Percentagem
- `‰`	Permilagem
- `∑`	Soma de
- `≠`	Diferente

In [3]:
!pip3 install pandas numpy

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
rows_to_keep = [4, 6]
rows_to_keep += range(8, 20)
rows_to_keep

### Load the data, drop all the non used rows, 

In [ ]:
rows_to_keep = [4, 6]
rows_to_keep += range(8, 19)
data = pd.read_csv(
    "./Dossier/Labs/Data/ine_principais_culturas_agricolas.csv",
    delimiter=";",
    encoding="latin-1",
    header=[1, 3],
    skiprows=lambda x: x not in rows_to_keep,
)

In [ ]:
data.drop([0], inplace=True)
data

## Reset of the Index as Region with just the region codes

In [ ]:
data[["Region", "Region_names"]] = data[data.columns[0]].str.split(
    ":",
    expand=True,
)
region_names = data["Region_names"].to_list()
data.set_index("Region", inplace=True)
columns_to_rem = [0, -1, -2]
data.drop(data.columns[columns_to_rem], axis=1, inplace=True)
data

## Redo the headers

In [ ]:
c0 = data.columns.get_level_values(0).to_series()
c1 = list(data.columns.get_level_values(1).to_series())
c0 = c0.mask(lambda c: c.str.startswith("Unnamed")).ffill()
c0

data.columns = [c0, c1]
data

## Do some exploratory analisys

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
c0.head()

In [ ]:
# data.columns.names(["Year", "Product"])

In [ ]:
# data.head()
data.tail()

## Start doing some replacements

In [ ]:
data.replace({'x':np.nan,'x x':np.nan,'- -':0,'nan':np.nan},inplace=True)
for column in data:
    data[column]=data[column].map(lambda x: str(x).rstrip(' &'))
    data[column]=data[column].map(lambda x: str(x).rstrip(' *'))

In [ ]:
for column in data:
    data[column]=pd.to_numeric(data[column],'coerce')

## Explore some sub tables

In [ ]:
trigo_data = data.xs("Trigo", axis=1, level=1)
trigo_data

In [ ]:
data.unstack()  # Show the df as a multiindex series with 3 index

In [ ]:
data["2021"]

## Creating a few graphs
- https://www.w3schools.com/python/pandas/pandas_plotting.asp
Mas que raio lisboa tem a mania que é grande ... alterar de "Área metropolitana de Lisboa" para "Lisboa" apenas ... poderei "Alfacinhas"

In [ ]:
data.columns[0]

In [ ]:
region_names[4] = "Lisboa"

In [ ]:
data[data.columns[1:10]] = data[data.columns[1:10]].astype(float)
data[data.columns[1:10]].plot(figsize=(16, 6))
plt.title("Quantity of Produced agriculture products by region")
plt.xlabel("Regions")
plt.ylabel("Quantity produced in kg/ ha")
plt.grid()
plt.xticks(range(len(region_names)), region_names)
plt.show()

In [ ]:
data.T

In [ ]:
data_norte_trigo.keys()

In [ ]:
# for key in ["Trigo mole", "Trigo duro"]:
#     plt.hist(data_norte_trigo[key], 10, label=key, alpha = 0.7)
# plt.grid()
# plt.title("Histogram of Trigo products in the north")
# plt.xlabel("Quantity produced in kg/ ha")
# plt.ylabel("Amount")
# plt.legend()
# plt.xticks(rotation='vertical')
# plt.show()

- Shet 0 is metadata
- Shet 1 is tmin
- Shet 2 is tmax
- Shet 3 is Pluviosity

In [ ]:
excel_list = [11, 132, 320, 571, 554]
code_list = [11, 16, 17, 18, 15]

In [ ]:
def add_extra_col(type_data, k):
    index_describe = 2
    if k == 3:
        index_describe = 7
        k = 0
    elif k == 4:
        index_describe = 3
        k = 1


    for k2, excel_code in enumerate(excel_list):
        code = code_list[k2]
        meteo2 = pd.read_excel(
            f"./Dossier/Labs/Data/IPMA/{excel_code}-tx-tn-prec.xlsx", sheet_name=(k + 1)
        )
        meteo2.dropna(axis=0, how="any", inplace=True)
        meteo2 = meteo2[meteo2.year > 1986]
        meteo2.set_index("year", inplace=True)
        meteo2.index.names = ["Region"]
        data_prec = meteo2.transpose().describe()
        indexNamesArr = data_prec.index.values
        indexNamesArr[index_describe] = code
        for k_name in indexNamesArr:
            if k_name != code:
                data_prec.drop(index=k_name, inplace=True)

        years_list = data_prec.keys()
        years_list
        header = [years_list, [type_data] * len(years_list)]
        data_prec.columns = header
        data_f = data_prec if k2 == 0 else pd.concat([data_f, data_prec])
    return data_f

In [ ]:
ipma = {
    type_data: add_extra_col(type_data, k)
    for k, type_data in enumerate(["Tmin", "Tmax", "Prec", "TmaxA", "TminA"])
}

In [ ]:
ipma

In [ ]:
meteo = pd.concat([ipma["Tmin"], ipma["Tmax"], ipma["Prec"], ipma["TminA"], ipma["TmaxA"]], axis=1)
meteo = meteo.reindex(sorted(meteo.columns), axis=1)
meteo

In [ ]:
meteo

In [ ]:
data

In [ ]:
data = data.stack(level=0)
data

In [ ]:
meteo= meteo.stack(level=0)
meteo

In [ ]:
meteo.index.names = ["Region", "Year"]

In [ ]:
meteo

In [ ]:
filter_col = [col for col in data if col.startswith("Trigo")]
data_norte_trigo = {
    "year": [],
}
for k_col, col in enumerate(filter_col):
    if col not in data_norte_trigo:
        data_norte_trigo[col] = []
    for elem in data[col]["PT"]:
        data_norte_trigo[col].append(elem)
        # if k_col == 0:
        #     data_norte_trigo["year"].append(elem[0])
    # region_year = col.index
    # if region_year[1] not in data_norte_trigo["year"]:
    #     data_norte_trigo["year"].append(region_year[1])

In [ ]:
data_norte_trigo

In [ ]:
filter_col = [col for col in data if col.startswith("Trigo")]
filter_col

In [ ]:
data_norte_trigo

In [ ]:
year_list = [tuple_list[1] for tuple_list in data.index]
year_list

In [ ]:
plt.figure(figsize=(16, 8))
for nkey, key in enumerate(data_norte_trigo):
    if nkey > 0:
        plt.plot(year_list, data_norte_trigo[key], label=key, marker="o")
plt.grid()
plt.title("Quantity of Trigo products by year in the north")
plt.xlabel("Years")
plt.ylabel("Quantity produced in kg/ ha")
plt.legend()
plt.xticks(rotation="vertical")
plt.show()

Dar replace the `dddd &` ou `dddd *`  1001 columns